In [1]:
import h5py
import pandas as pd
import numpy as np

# Replace with your HDF5 file path
hdf5_file_path = "C:/Users/BPIN/Desktop/reva_hackathon-23/hdf5greenland.h5"

# Open the HDF5 file for reading
with h5py.File(hdf5_file_path, 'r') as file:
    # Access the VH backscatter dataset
    vh_backscatter_dataset = file['science/LSAR/SLC/swaths/frequencyA/VH']
    # Read the data and take the absolute value to get amplitude
    vh_backscatter_amplitude = np.abs(vh_backscatter_dataset[:])
    
    # Access the VV backscatter dataset
    vv_backscatter_dataset = file['science/LSAR/SLC/swaths/frequencyA/VV']
    # Read the data and take the absolute value to get amplitude
    vv_backscatter_amplitude = np.abs(vv_backscatter_dataset[:])
    
    # Access the incidence angle dataset
    incidence_angle_dataset = file['science/LSAR/SLC/metadata/geolocationGrid/incidenceAngle']
    # Read the data and convert to degrees if necessary
    incidence_angle_data = np.array(incidence_angle_dataset[:])
    
    # Flatten the 2D arrays to 1D arrays if they are not already
    vh_backscatter_flat = vh_backscatter_amplitude.flatten()[:1000]
    vv_backscatter_flat = vv_backscatter_amplitude.flatten()[:1000]
    incidence_angle_flat = incidence_angle_data.flatten()[:1000]

    # Convert VH and VV backscatter amplitude to dB
    vh_backscatter_db = 10 * np.log10(vh_backscatter_flat**2)
    vv_backscatter_db = 10 * np.log10(vv_backscatter_flat**2)

# Now, let's create the pandas DataFrame for the first 1000 entries
# Soil moisture content would be added later, so we initialize it with NaNs for now
soil_moisture_content = np.full(vh_backscatter_db.shape, np.nan)

df = pd.DataFrame({
    'VH_Backscatter_dB': vh_backscatter_db,
    'VV_Backscatter_dB': vv_backscatter_db,
    'Incidence_Angle_degrees': incidence_angle_flat,  # Convert to degrees if necessary
    'Soil_Moisture_Content': soil_moisture_content
})

# Display the first few rows of the DataFrame
print(df.head())


ModuleNotFoundError: No module named 'h5py'

In [2]:
# Initialize arrays to hold the data
vh_backscatter = []
vv_backscatter = []
incidence_angle = []

# Open the HDF5 SAR data file
with h5py.File(hdf5_file_path, 'r') as file:
    # Read VH and VV backscatter data
    vh_backscatter = file['science/LSAR/SLC/swaths/frequencyA/VH'][:1000, :]  # Adjust indices as needed
    vv_backscatter = file['science/LSAR/SLC/swaths/frequencyA/VV'][:1000, :]  # Adjust indices as needed
    
    # Read incidence angle data
    incidence_angle = file['science/LSAR/SLC/metadata/geolocationGrid/incidenceAngle'][:1000, :]  # Adjust indices as needed

# Converting amplitude to dB
vh_backscatter_db = 10 * np.log10(np.abs(vh_backscatter))
vv_backscatter_db = 10 * np.log10(np.abs(vv_backscatter))

# Making sure incidence angle is in degrees if it's not already
incidence_angle_degrees = np.rad2deg(incidence_angle)

# Output the shapes of the arrays as a sanity check
(vh_backscatter_db.shape, vv_backscatter_db.shape, incidence_angle_degrees.shape)


NameError: name 'h5py' is not defined

In [10]:
# Assuming incidence_angle has the shape (6, 1085, 225) which could be 6 different frames or something similar
# You need to find the correct incidence angle data that corresponds to your backscatter data
# This is a placeholder operation, you will need to adjust this based on how your data is structured
incidence_angle_corrected = incidence_angle[0]  # This is just an example, you'll need to adjust it as per your data structure

# Now, reshape or interpolate the incidence angle to match the backscatter array's dimensions
# For this example, I'm using a simple repeat operation, but you'll likely need something more sophisticated
incidence_angle_repeated = np.repeat(incidence_angle_corrected, 2581/225, axis=1)[:1000, :]

# Sanity check for shapes
(vh_backscatter_db.shape, incidence_angle_repeated.shape)


((1000, 2581), (1000, 2475))

In [11]:
from scipy.interpolate import interp2d

# Placeholder coordinates, you'll need actual coordinates corresponding to your data
x = np.linspace(0, 1, incidence_angle_corrected.shape[1])
y = np.linspace(0, 1, incidence_angle_corrected.shape[0])
f = interp2d(x, y, incidence_angle_corrected, kind='linear')

# New coordinates at which to interpolate
new_x = np.linspace(0, 1, 2581)
new_y = np.linspace(0, 1, 1000)

# Perform the interpolation
incidence_angle_interpolated = f(new_x, new_y)

# Check the new shapes
(vh_backscatter_db.shape, incidence_angle_interpolated.shape)


C:\Users\BPIN\AppData\Local\Temp\ipykernel_3524\642689827.py:6: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  f = interp2d(x, y, incidence_angle_corrected, kind='linear')
C:\Users\BPIN\AppData\Local\Temp\ipykernel_3524\642689827.py:13: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev`

((1000, 2581), (1000, 2581))